### Test function to clean & un-split city names data, from the scraped rental listings CSV files:

In [2]:
# imports-- file processing
import os
import glob

# data analysis libraries & SQL libraries
import numpy as np
import pandas as pd
# SQL ODBC for API connection between Python & SQL Server
import pyodbc
import sqlalchemy as sa

In [4]:
def recursively_import_all_CSV_and_concat_to_single_df(parent_direc, fn_regex=r'*.csv'):
    """Recursively search parent directory, and look up all CSV files.
    Then, import all CSV files to a single Pandas' df using pd.concat()"""
    path =  parent_direc # specify parent path of directories containing the scraped rental listings CSV data -- NB: use raw text--as in r'path...', or can we use the double-back slashes to escape back-slashes??
    df_concat = pd.concat((pd.read_csv(file) for file in glob.iglob(
        os.path.join(path, '**', fn_regex), 
        recursive=True)), ignore_index=True)  # os.path.join helps ensure this concatenation is OS independent
    return df_concat

## Import Dataset
# import all scraped SF bay area rental listings data
scraped_data_path = r"D:\\Coding and Code projects\\Python\\craigslist_data_proj\\CraigslistWebScraper\\scraped_data\\sfbay"

df = recursively_import_all_CSV_and_concat_to_single_df(scraped_data_path)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18645 entries, 0 to 18644
Data columns (total 48 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   listing_urls             18645 non-null  object 
 1   ids                      17236 non-null  float64
 2   sqft                     13209 non-null  float64
 3   cities                   17219 non-null  object 
 4   prices                   17227 non-null  object 
 5   bedrooms                 17184 non-null  float64
 6   bathrooms                17184 non-null  object 
 7   attr_vars                17220 non-null  object 
 8   listing_descrip          17220 non-null  object 
 9   date_of_webcrawler       17236 non-null  object 
 10  kitchen                  17220 non-null  float64
 11  date_posted              17220 non-null  object 
 12  region                   18645 non-null  object 
 13  sub_region               18645 non-null  object 
 14  cats_OK               

In [27]:
# get copy so we can test dataset without having to repeatedly re-load dataset
df1 = df.copy()

In [28]:
def clean_split_city_names(df, address_critera: list, neighborhood_criteria:list, split_city_delimiters: list):
    """Clean city names data in several ways:
    a.) Remove extraneous address & neighborhood data placed in the city names HTML object, such as 'Rd', 'Blvd', or 'Downtown'.
    b.) Unsplit city names data that are split via ',' & '/' delimiters.
    c.) Replace abbreviated or mispelled city names.
    d.) Remove any digits/integers within the city names data--ie, by using a '\d+' regex as the argument of str.replace() and replace it with empty strings.
    e.) Remove any city names records thast are left with merely empty strings (ie, the other steps removed all data for that given cities record).
    f.) Remove any whitespace to avoid the same city names from being treated as different entities by Pandas, Python, or SQL. 
    g.) Also, use str.capwords() to capitalize words (ie, excluding apostrophes)."""
    addr_criteria = '|'.join(address_critera) # Join pipe ('|') symbols to address list so we can str.split() on any one of these criteria (ie, 'or' condition splitting on each element separated by pipes):
    # specify extraneous neighborhood criteria we should also remove from col
    nbhood_criteria = '|'.join(neighborhood_criteria) # remove neighborhood names as well as state abbreviation (shown on website as 'Ca') that is shown without the usual comma delimiter!
    # b.) specify delimiters we need to refer to un-split city names:
    split_city_delimiters = '|'.join(split_city_delimiters) # join pipes to delimiters so we can use str.split() based on multiple 'or' criteria simultaneously
    # clean city names data by removing extraneous address & neighborhood data, and unsplitting city names based on ',' & '\' delimiters
    df['cities'] =  df['cities'].str.split(addr_criteria).str[-1].str.replace(nbhood_criteria, '', case=True).str.lstrip()
    df['cities'] = df['cities'].str.split(split_city_delimiters).str[0] #unsplit city names based on comma or forward-slash delimiters
    # c.) replace specific abbreviated or mispelled city names:
    df = df.replace({'cities':{'Rohnert Pk':'Rohnert Park', 'Hillsborough Ca': 'Hillsborough', 'South Sf': 'South San Francisco', 
    'East San Jose':'San Jose', 'Vallejo Ca':'Vallejo', 'Westgate On Saratoga .':'San Jose', 'Bodega':'Bodega Bay', 'Briarwood At Central Park':'Fremont', 'Campbell Ca': 'Campbell', 
    'Almaden':'San Jose', '.':'', 'Bend' :'', 'Ketchum':'', 'Baypoinr':'', ' Ca':'', 'Quito': '', 
    'Lake County''', 'Redding':'', 'Redwood Shores':'Redwood City'}}) 
    # d.) Remove digits/integers from cities column records:
    df['cities'] = df['cities'].str.replace('\d+', '')  # remove any digits by using '/d+' regex to look up digits, and then replace with empty string
    # e.) Remove any rows that have empty strings or null values for cities col (having performed the various data filtering and cleaning above)
    df = df[df['cities'].str.strip().astype(bool)] # remove rows with empty strings (ie, '') for cities col 
    df = df.dropna(subset=['cities']) # remove any remaining 'cities' null records
    # f.) Remove whitespace
    df['cities'] = df['cities'].str.strip() 
    # g.) capitalize the city names using str.capwords() 
    df['cities'] = df['cities'].str.split().apply(lambda x: [val.capitalize() for val in x]).str.join(' ')
    return df



# specify various address and street name that we need to remove from the city names 
address_criteria = ['Boulevard', 'Blvd', 'Road', 'Rd', 'Avenue', 'Ave', 'Street', 'St', ' Dr', 'Drive', 'Real', 'E Hillsdale Blvd'] 
# specify various extraneous neighborhood names such as 'Downtown' 
neighborhood_criteria = ['Downtown', 'Central/Downtown', 'North', 'California', 'Ca', 'Ca.', 'Bay Area', 'St. Helena', 'St', 'nyon', 'Jack London Square', 'Walking Distance To', 'El Camino', 'Mendocino County', 'San Mateo County', 'Alameda County', 'Rio Nido Nr', 'Mission Elementary', 'Napa County', 'Golden Gate', 'Jennings', 'South Lake Tahoe', 'Tahoe Paradise', 'Kingswood Estates', 'South Bay', 'Skyline', 'San Antonio Tx', 'East Bay', 'Morton Dr']
# specify what delimiters we want to search for to unsplit the split city names data:
split_city_delimiters =  [',', '/']

# clean city names data:
df1 = clean_split_city_names(df1, neighborhood_criteria, address_criteria, split_city_delimiters)
# sanity check
df1.cities.value_counts().tail(10)


d:\Coding and Code projects\Python\craigslist_data_proj\CraigslistWebScraper\craigslist_venv\lib\site-packages\ipykernel_launcher.py:16: FutureWarning: The default value of regex will change from True to False in a future version.
  app.launch_new_instance()
d:\Coding and Code projects\Python\craigslist_data_proj\CraigslistWebScraper\craigslist_venv\lib\site-packages\ipykernel_launcher.py:24: FutureWarning: The default value of regex will change from True to False in a future version.


Alamo               1
Summerville         1
San Lorenzo         1
Antioch             1
Mariner's Island    1
Montara             1
Windsor             1
Portola Valley      1
Napa                1
Rohnert Park        1
Name: cities, dtype: int64

In [16]:
df1['cities'].value_counts().tail(60)

West Marin           26
Boulder Creek        26
Woodside             26
Soquel               20
Half Moon Bay        19
San Anselmo          18
Tiburon              17
Hollister            14
Larkspur             14
Brisbane             13
Greenbrae            13
Lake County          12
Corte Madera         10
Occidental            9
Redwood Shores        9
Saratoga              7
Atherton              7
Corralitos            6
Kentfield             6
Forestville           5
Westbrae              5
Elmwood               5
Pinole                4
El Sobrante           3
Kenwood               3
Tracy                 3
Kelseyville           3
Pleasure Point        2
Felton                2
Pleasant Hill         2
Guerneville           2
Bodega Bay            2
San Ramon             2
Redding               2
East Foothills        1
Rockridge             1
Los Banos             1
San Juan Bautista     1
Deer Park             1
Pleasanton            1
Orinda                1
San Pablo       

In [18]:
import os
os.getcwd()

'd:\\Coding and Code projects\\Python\\craigslist_data_proj\\CraigslistWebScraper\\Rentals'

In [17]:
# export to csv
df1.to_csv('cleaned_cities_demo.csv')

In [26]:
df.loc[df['ids'] == 7400147944]

,listing_urls,ids,sqft,cities,prices,bedrooms,bathrooms,attr_vars,listing_descrip,date_of_webcrawler,...,is_furnished,attached_garage,detached_garage,carport,off_street_parking,no_parking,EV_charging,air_condition,no_smoking,Unnamed: 0
9398,https://sfbay.craigslist.org/pen/apa/d/san-car...,7.400148e+09,NaN,San Carlos,"3,600",1.0,1.0,flooring: carpet\nfurnished\napartment\nlaundr...,"beautiful corner, spacious, very large 1 bedro...",2021-11-09,...,1,1,0,0,0,0,0,0,1,NaN
9909,https://sfbay.craigslist.org/pen/apa/d/redwood...,7.400148e+09,NaN,San Carlos,"3,600",1.0,1,flooring: carpet\nfurnished\napartment\nlaundr...,"beautiful corner, spacious, very large 1 bedro...",2021-11-18,...,1,1,0,0,0,0,0,0,1,NaN
